# Data Binding

In [8]:
# 1. Gerekli Kütüphaneleri Kur
!pip install -U chromadb langchain-huggingface PyPDF2 groq yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 18.7 MB/s e

In [9]:
import os
import shutil
import re
from google.colab import drive
from PyPDF2 import PdfReader
import chromadb
from langchain_huggingface import HuggingFaceEmbeddings

# ============================================================
# 0) GOOGLE DRIVE BAĞLANTI (SENİN DRIVE PATH'İN)
# ============================================================
drive.mount("/content/drive")

SOURCE_DIR = "/content/drive/My Drive/agent_real"
DB_PATH    = "/content/drive/My Drive/agent_real/chroma_db_persistent"

print("📂 SOURCE_DIR içerik:", os.listdir(SOURCE_DIR))

# ============================================================
# 1) ESKİ / BOZUK DB'Yİ TEMİZLE
# ============================================================
shutil.rmtree(DB_PATH, ignore_errors=True)
os.makedirs(DB_PATH, exist_ok=True)
print("🧹 Eski ChromaDB temizlendi.")

# ============================================================
# 2) CHROMA INDEXER
# ============================================================
class ChromaIndexer:
    def __init__(self, db_path):
        self.client = chromadb.PersistentClient(path=db_path)

        self.embedding_fn = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-L6-v2"
        )

        self.collection = self.client.get_or_create_collection(
            name="eurobond_knowledge"
        )

        print("✅ Chroma collection hazır.")

    def extract_text(self, file_path):
        ext = os.path.splitext(file_path)[1].lower()
        text = ""

        try:
            if ext == ".pdf":
                reader = PdfReader(file_path)
                for page in reader.pages:
                    content = page.extract_text()
                    if content:
                        text += content + "\n"

            elif ext == ".txt":
                with open(file_path, "r", encoding="utf-8") as f:
                    text = f.read()

        except Exception as e:
            print(f"❌ Okuma hatası ({file_path}): {e}")

        return text

    def chunk_text(self, text):
        return [
            s.strip()
            for s in re.split(r'(?<=[.!?])\s+', text)
            if len(s.strip()) > 40
        ]

    def process_and_save(self, files):
        total_chunks = 0

        for fname in files:
            path = os.path.join(SOURCE_DIR, fname)

            if not os.path.exists(path):
                print(f"⚠️ Dosya yok: {fname}")
                continue

            print(f"📄 İşleniyor: {fname}")
            raw = self.extract_text(path)

            if not raw.strip():
                print(f"⚠️ Boş içerik: {fname}")
                continue

            chunks = self.chunk_text(raw)

            docs, metas, ids = [], [], []

            for i, ch in enumerate(chunks):
                docs.append(ch)
                metas.append({"source": fname})
                ids.append(f"{fname}_{i}_{abs(hash(ch))}")

            if docs:
                self.collection.add(
                    documents=docs,
                    metadatas=metas,
                    ids=ids
                )
                total_chunks += len(docs)
                print(f"✅ {fname} → {len(docs)} chunk")

        print(f"\n🚀 Indexleme tamamlandı | Toplam chunk: {total_chunks}")
        print(f"📊 Chroma toplam kayıt: {self.collection.count()}")

# ============================================================
# 3) ÇALIŞTIR
# ============================================================
indexer = ChromaIndexer(DB_PATH)

files_to_index = [
    "AKE-Eurobond.pdf",
    "eurobond-bulten.pdf",
    "Eurobonds.pdf",
    "gedik.txt",
    "rates.txt"
]

indexer.process_and_save(files_to_index)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📂 SOURCE_DIR içerik: ['Eurobonds.pdf', 'eurobond-bulten.pdf', 'gedik.txt', 'rates.txt', 'AKE-Eurobond.pdf']
🧹 Eski ChromaDB temizlendi.
✅ Chroma collection hazır.
📄 İşleniyor: AKE-Eurobond.pdf
✅ AKE-Eurobond.pdf → 20 chunk
📄 İşleniyor: eurobond-bulten.pdf
✅ eurobond-bulten.pdf → 7 chunk
📄 İşleniyor: Eurobonds.pdf
✅ Eurobonds.pdf → 200 chunk
📄 İşleniyor: gedik.txt
✅ gedik.txt → 36 chunk
📄 İşleniyor: rates.txt
✅ rates.txt → 37 chunk

🚀 Indexleme tamamlandı | Toplam chunk: 300
📊 Chroma toplam kayıt: 300


# GROQ

In [3]:
!pip install groq
import groq

print(groq.__version__)

1.0.0


In [4]:
# =========================
# PART 1 — Environment & API Setup (Eurobond)
# =========================
import re
import math
from collections import Counter

import yfinance as yf
from google.colab import userdata
from groq import Groq

groq_api_key = userdata.get('GROQ_API_KEY')
client = Groq(api_key=groq_api_key)

In [5]:
#model baglanti testi
from google.colab import userdata

from groq import Groq

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Hello' how are you?",
        }
    ],
    model="llama-3.3-70b-versatile" # "llama-3.1-70b-versatile", # llama3-70b-8192
)

print(chat_completion.choices[0].message.content)

Hello. I'm doing well, thanks for asking. I'm a large language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help with any questions or tasks you may have. How can I assist you today?


In [14]:
import re
import yfinance as yf
from google.colab import userdata
from groq import Groq
import chromadb
from chromadb.utils import embedding_functions

# ============================================================
# 1) API AYARLARI
# ============================================================
try:
    groq_api_key = userdata.get('GROQ_API_KEY')
    client = Groq(api_key=groq_api_key)
except Exception:
    raise RuntimeError("GROQ_API_KEY bulunamadı. Colab Secrets'ı kontrol et.")

# ============================================================
# 2) CHROMADB EUROBOND RAG TOOL (HAM CONTEXT DÖNDÜRÜR)
# ============================================================
CHROMA_PERSIST_DIR = "/content/drive/My Drive/agent_real/chroma_db_persistent"
CHROMA_COLLECTION_NAME = "eurobond_knowledge"

chroma_client = chromadb.PersistentClient(path=CHROMA_PERSIST_DIR)
chroma_collection = chroma_client.get_collection(
    name=CHROMA_COLLECTION_NAME
)

def eurobond_search_tool(query: str) -> str:
    """
    Retrieves Eurobond-related information from ChromaDB as RAW CONTEXT.
    This tool does NOT answer the question.
    """
    print(f"\n🔍 [EUROBOND RAG] Searching for: '{query}'")

    try:
        results = chroma_collection.query(
            query_texts=[query],
            n_results=5,
            include=["documents", "metadatas"]
        )
    except Exception as e:
        return f"Internal database error: {e}"

    documents = (results.get("documents") or [[]])[0]
    metadatas = (results.get("metadatas") or [[]])[0]

    if not documents:
        return "Internal database has no information on this Eurobond-related topic."

    context = []
    for doc, meta in zip(documents, metadatas):
        source = meta.get("source", "unknown source")
        context.append(f"[Source: {source}]\n{doc}")

    return "Internal Doc Results:\n\n" + "\n\n".join(context)

# ============================================================
# 3) DİĞER TOOLS (ReAct uyumlu)
# ============================================================
def calculator(expression: str) -> str:
    """Calculates a mathematical expression."""
    try:
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters."
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

def last_close_price(ticker: str):
    """Returns last close price of a stock."""
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()
    return info.get("previousClose", "N/A")

tools = [
    eurobond_search_tool,
    calculator,
    last_close_price
]

tools_with_desc = "\n\n".join([tool.__name__ + (tool.__doc__ or "") for tool in tools])
known_actions = dict([(tool.__name__, tool) for tool in tools])

# ============================================================
# 4) SYSTEM PROMPT (AJAN BEYNİ)
# ============================================================
system_prompt = f"""
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.

GUIDELINES:
1. Use 'eurobond_search_tool' to retrieve Eurobond-related knowledge as RAW CONTEXT.
   - This tool does NOT provide final answers.
   - You must synthesize the answer yourself.
2. The Eurobond knowledge base is built from publicly available PDF documents.
3. If no relevant information is found, explicitly say so and DO NOT hallucinate.
4. Use 'calculator' for numerical calculations only.
5. Use 'last_close_price' only for public market price questions.

DECISION RULES:
- If a retrieved document explicitly contains the requested numeric value
  (e.g., coupon rate, maturity, yield),
  you MUST stop searching and proceed to calculation.
- Do NOT perform additional searches once sufficient information is found.
- A maturity date of January 2030 corresponds to TR2030.

IMPORTANT ACTION FORMAT RULES (MUST FOLLOW EXACTLY):
- When you decide to use a tool, you MUST write the action in this exact format:

Action: tool_name: tool_input

- Do NOT use hyphens (-) in the Action line.
- Do NOT add explanations or extra text in the Action line.
- Use ONE Action per response only.
- After writing an Action line, stop and wait for PAUSE.

FINAL ANSWER FORMAT RULES:
- When a coupon rate and an investment amount are both requested,
  you MUST present the final answer in the following format:

Coupon rate: <percentage>
Annual coupon income (USD <investment_amount>): <amount>

- Always include BOTH the coupon rate and the calculated income.
- Do NOT omit the coupon rate in the final answer.


Your available actions are:

{tools_with_desc}
""".strip()


# ============================================================
# 5) AGENT (hocanın koduna sadık)
# ============================================================
class Agent:
    def __init__(self, system=""):
        self.messages = []
        if system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        result = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=self.messages,
            temperature=0,
            stop=["PAUSE", "Observation:", "<|"]
        )
        return result.choices[0].message.content

# ============================================================
# 6) ORKESTRASYON DÖNGÜSÜ (ReAct)
# ============================================================
action_re = re.compile(r'^Action: (\w+): (.*)$')

def query(question, max_turns=10, log_file="react_agent_logs.txt"):
    react_agent = Agent(system_prompt)
    next_prompt = question

    # Optional: Log the start of a new question
    with open(log_file, "a") as f:
        f.write(f"\n{'='*50}\nQUESTION: {question}\n{'='*50}\n")

    print(f"\n{'='*50}\nQUESTION: {question}\n{'='*50}")

    for _ in range(max_turns):
        result = react_agent(next_prompt)
        print(result)

        # Log the agent's thought process
        with open(log_file, "a") as f:
            f.write(result + "\n")

        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

        if actions:
            action, action_input = actions[0].groups()
            print(f"=> Running {action} with input: {action_input}")

            if action not in known_actions:
                raise Exception(f"Unknown action: {action}")

            observation = known_actions[action](action_input)
            print(f"Observation: {observation}")

            # Log the observation from the tool
            with open(log_file, "a") as f:
                f.write(f"Observation: {observation}\n")

            next_prompt = f"Observation: {observation}"
        else:
            if "Answer:" in result:
                final_ans = result.split("Answer:")[-1].strip()
                print("\n--- FINAL ANSWER ---")
                print(final_ans)

                # Log the final answer
                with open(log_file, "a") as f:
                    f.write(f"\n--- FINAL ANSWER ---\n{final_ans}\n")
            return

# =========================
# SCENARIO-BASED AGENT BENCHMARK
# =========================

benchmark_cases = [
    # --- SECTION 1: COMPARISON & RANKING ---
    {"id": "C1", "name": "Comparison", "question": "Calculate the spread in basis points between the Turkey 2036 buy yield (6.41%) and the US 10Y yield (4.13%) as of December 24, 2025."},
    {"id": "C2", "name": "Comparison", "question": "Compare the 5-year CDS spreads of Turkey, Brazil, and South Africa. Which country has the highest risk premium according to the data?"},
    {"id": "C3", "name": "Comparison", "question": "Identify which country's 10-year Eurobond yield saw the largest basis point decrease between November 24 and December 24, 2025."},
    {"id": "C4", "name": "Comparison", "question": "Rank the Turkey 2027, 2029, and 2047 Eurobonds by their buy yields on 24/12/2025 from lowest to highest."},
    {"id": "C5", "name": "Comparison", "question": "Between ISIN JS900123AL40 and JS900123BB58, which bond has the longer remaining term to maturity from the bulletin date?"},
    {"id": "C6", "name": "Comparison", "question": "Determine the difference in coupon rates between the Akbank 2026 (6.800%) and Vakifbank 2026 (6.500%) USD Eurobonds."},

    # --- SECTION 2: LOGIC & MATH ---
    {"id": "M1", "name": "Logic & Math", "question": "If a full taxpayer earns 450,000 TRY in Eurobond interest in 2025, by how much does this exceed the 330,000 TRY declaration threshold?"},
    {"id": "M2", "name": "Logic & Math", "question": "If the AKE fund total net value is 1.5 billion USD, what is the minimum dollar amount that must be invested in USD-denominated Turkish Eurobonds per the 80% rule?"},
    {"id": "M3", "name": "Logic & Math", "question": "Calculate the percentage difference between the Treasury Sukuk coupon (7.250%) and the Sisecam 2026 coupon (6.950%)."},
    {"id": "M4", "name": "Logic & Math", "question": "Calculate the total annual interest income in USD for a client holding 250,000 USD nominal of the Turkey 2047 bond (5.750% coupon)."},
    {"id": "M5", "name": "Logic & Math", "question": "If the maximum 1% commission is charged on a 15,000 USD nominal Eurobond transaction, what is the total commission fee?"},
    {"id": "M6", "name": "Logic & Math", "question": "Find the average 5-year CDS spread of Brazil and South Africa as of 24/12/2025 and determine how much lower it is than Turkey's CDS."},

    # --- SECTION 3: MULTI-HOP (Cross-Document) ---
    {"id": "H1", "name": "Multi-Hop", "question": "Based on the T+2 settlement rule in the HSBC document, if a trade is executed on the Gedik bulletin date (Dec 25, 2025), what is the specific settlement date?"},
    {"id": "H2", "name": "Multi-Hop", "question": "Find the 2025 tax declaration threshold in TRY and convert it to USD using an assumed exchange rate of 1 USD = 35 TRY."},
    {"id": "H3", "name": "Multi-Hop", "question": "Identify all USD government bonds in the 'Daily Transactions' table with a Moody's rating of 'Ba3' and a buy yield exceeding 6%."},
    {"id": "H4", "name": "Multi-Hop", "question": "Locate the coupon rate for Arcelik's EUR bond and determine its withholding tax rate based on the HSBC tax summary for individuals."},
    {"id": "H5", "name": "Multi-Hop", "question": "Compare the AKE fund's annual management fee (2.00%) with the maximum transaction commission rate (1.00%); which is higher?"},
    {"id": "H6", "name": "Multi-Hop", "question": "Find the bank bond with the highest coupon rate in the bulletin and identify which risk profile ('Low' to 'Very High') it belongs to according to the product guidelines."},

    # --- SECTION 4: FALLBACK & HALLUCINATION ---
    {"id": "F1", "name": "Fallback", "question": "What is the coupon rate of the Turkey 2062 Eurobond? If the data is missing, state it and provide the current stock price of Apple (AAPL)."},
    {"id": "F2", "name": "Fallback", "question": "Search the documents for 'Social Impact Bonds'. If not found, provide the definition of 'Country Risk' from the HSBC document instead."},
    {"id": "L1", "name": "Hallucination", "question": "Check the bulletin for the current trading price of 'Gedik Yatirim' stock. Do not provide a price if it is not explicitly in the text."},
    {"id": "L2", "name": "Hallucination", "question": "Confirm if the documents state that Turkish Treasury Eurobonds are covered by the 'Investor Compensation Center' up to 100,000 EUR."},
    {"id": "L3", "name": "Hallucination", "question": "Find the 'Loyalty Discount' for long-term holders in the HSBC fee section. If it does not exist, state the standard maximum commission."},

    # --- SECTION 5: TEMPORAL ---
    {"id": "T1", "name": "Temporal", "question": "Using the 'Active Turkey Bonds (USD)' table, describe the yield trend for the Turkey 2029 bond between 24/11/2025 and 24/12/2025."},
    {"id": "T2", "name": "Temporal", "question": "Calculate the number of years between the establishment of the Ak Portfoy AKE fund (Dec 2, 2010) and the bulletin date (Dec 25, 2025)."},
    {"id": "T3", "name": "Temporal", "question": "List all USD government bonds in the daily transactions table that have a maturity date in the year 2034 or later."},

    # --- SECTION 6: CATEGORICAL FILTERING ---
    {"id": "S1", "name": "Filtering", "question": "Extract all EUR-denominated bonds from the dataset and list their issuers and coupon rates."},
    {"id": "S2", "name": "Filtering", "question": "List all 'Subordinated' (Sermaye Benzeri) bonds found in the bulletin and include their respective bank names."},
    {"id": "S3", "name": "Filtering", "question": "Filter the list of government bonds to show only those with a minimum nominal transaction requirement of 200,000 USD."},
    {"id": "S4", "name": "Filtering", "question": "Identify which 'Non-Financial' private sector bonds have a maturity date specifically in the year 2026."},
    {"id": "S5", "name": "Filtering", "question": "List every bond in the 'Daily Transactions' tables that currently has an 'Ask Price' (Alis Fiyati) above 105.000."},

    # --- SECTION 7: POLICY & RISK ---
    {"id": "R1", "name": "Policy", "question": "Explain why the HSBC document advises against Eurobonds for investors who do not have 'emergency cash' reserves."},
    {"id": "R2", "name": "Policy", "question": "Detail the specific restrictions mentioned regarding the sale of these investment products to residents of the USA and Canada."},
    {"id": "R3", "name": "Policy", "question": "According to the 'Right of Withdrawal' section, can an investor cancel a Eurobond purchase after it has been executed?"},
    {"id": "R4", "name": "Policy", "question": "Define 'Liquidity Risk' as described in the fundamental risks section of the Eurobond documentation."},
    {"id": "R5", "name": "Policy", "question": "What is the tax status of Eurobond interest income for a 'limited taxpayer' individual (Dar mukellef)?"},

    # --- SECTION 8: COMPLEX SCENARIOS ---
    {"id": "X1", "name": "Scenario", "question": "An investor wants a bond with a coupon of at least 9.0%. List all government and private options that meet this criteria."},
    {"id": "X2", "name": "Scenario", "question": "Calculate the total TRY value of one semi-annual coupon payment for a 20,000 USD nominal holding of the Turkey 2030 bond (11.875% coupon), using 1 USD = 35 TRY."},
    {"id": "X3", "name": "Scenario", "question": "Compare the 'Average Dirty Price' (Ortalama Kirli Fiyat) of the Turkey 2030 bond with the Turkey 2047 bond."},
    {"id": "X4", "name": "Scenario", "question": "Is the 'Credit Risk' stated to be higher for corporate bonds or government bonds according to the HSBC Risk Profile?"},
    {"id": "X5", "name": "Scenario", "question": "Under what specific condition can the purchase price of a Eurobond be adjusted for inflation for capital gains tax purposes?"},
    {"id": "X6", "name": "Scenario", "question": "Who is the legal issuer responsible for the principal and interest payments of the Eurobonds listed in the HSBC document?"},
    {"id": "X7", "name": "Scenario", "question": "Describe the tax consequences of transferring Eurobonds from an individual account to a joint account according to the policy."},
    {"id": "X8", "name": "Scenario", "question": "Find the 'Days to Next Coupon' (Kupona Kalan Gun) for the Turkey 2034 bond (ISIN ending in AT75) as of the bulletin date."},
    {"id": "X9", "name": "Scenario", "question": "Which regulatory bodies are mentioned as the authorities supervising HSBC Bank A.S.'s banking and capital market activities?"}
]

action_re = re.compile(r'^Action: (\w+): (.*)$')

react_agent = Agent(system_prompt)
react_agent("What is the coupon rate of TR2030 Eurobond?")



def run_agent_benchmark(log_file="react_agent_logs.txt"):
    print("\n" + "="*80)
    print("STARTING SCENARIO-BASED AGENT BENCHMARK")
    print("="*80)

    summary = []

    for case in benchmark_cases:
        print("\n" + "-"*80)
        print(f"Scenario {case['id']}: {case['name']}")
        print(f"Question: {case['question']}")

        # ✅ SAFE: expected_behavior optional
        expected = case.get("expected_behavior", [])
        if expected:
            print("Expected behavior:")
            for exp in expected:
                print(f"  - {exp}")
        else:
            print("Expected behavior: (not specified)")

        # Run the agent
        query(case["question"], log_file=log_file)

        summary.append({
            "scenario_id": case["id"],
            "name": case["name"],
            "question": case["question"],
            "expected_behavior": expected,
            "log_file": log_file
        })

    print("\n" + "="*80)
    print("BENCHMARK COMPLETE")
    print("="*80)

    return summary

run_agent_benchmark()




STARTING SCENARIO-BASED AGENT BENCHMARK

--------------------------------------------------------------------------------
Scenario C1: Comparison
Question: Calculate the spread in basis points between the Turkey 2036 buy yield (6.41%) and the US 10Y yield (4.13%) as of December 24, 2025.
Expected behavior: (not specified)

QUESTION: Calculate the spread in basis points between the Turkey 2036 buy yield (6.41%) and the US 10Y yield (4.13%) as of December 24, 2025.
Action: calculator: (6.41 - 4.13) * 100
=> Running calculator with input: (6.41 - 4.13) * 100
Observation: 228.00000000000003
The spread in basis points between the Turkey 2036 buy yield and the US 10Y yield as of December 24, 2025, is 228 basis points. 

Answer: The spread is 228 basis points.

--- FINAL ANSWER ---
The spread is 228 basis points.

--------------------------------------------------------------------------------
Scenario C2: Comparison
Question: Compare the 5-year CDS spreads of Turkey, Brazil, and South Afric

ERROR:yfinance:HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: GEDIK YATIRIM"}}}


Observation: N/A
The last close price of 'Gedik Yatirim' stock is not available.

--------------------------------------------------------------------------------
Scenario L2: Hallucination
Question: Confirm if the documents state that Turkish Treasury Eurobonds are covered by the 'Investor Compensation Center' up to 100,000 EUR.
Expected behavior: (not specified)

QUESTION: Confirm if the documents state that Turkish Treasury Eurobonds are covered by the 'Investor Compensation Center' up to 100,000 EUR.
Action: eurobond_search_tool: Turkish Treasury Eurobonds Investor Compensation Center coverage
=> Running eurobond_search_tool with input: Turkish Treasury Eurobonds Investor Compensation Center coverage

🔍 [EUROBOND RAG] Searching for: 'Turkish Treasury Eurobonds Investor Compensation Center coverage'
Observation: Internal Doc Results:

[Source: AKE-Eurobond.pdf]
The Fund invest sinTurkish Treasury and Corporate Eurobonds (USD)AKE -Eurobond (USD) Fixed Income Fund
Would you liketoadd 

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kdracekqfghadmpszepbh1r2` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 100000, Requested 710. Please try again in 10m13.44s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

# GPT 4o Mini Benchmark

In [15]:


benchmark_cases = [
    # --- SECTION 1: COMPARISON & RANKING ---
    {"id": "C1", "name": "Comparison", "question": "Calculate the spread in basis points between the Turkey 2036 buy yield (6.41%) and the US 10Y yield (4.13%) as of December 24, 2025."},
    {"id": "C2", "name": "Comparison", "question": "Compare the 5-year CDS spreads of Turkey, Brazil, and South Africa. Which country has the highest risk premium according to the data?"},
    {"id": "C3", "name": "Comparison", "question": "Identify which country's 10-year Eurobond yield saw the largest basis point decrease between November 24 and December 24, 2025."},
    {"id": "C4", "name": "Comparison", "question": "Rank the Turkey 2027, 2029, and 2047 Eurobonds by their buy yields on 24/12/2025 from lowest to highest."},
    {"id": "C5", "name": "Comparison", "question": "Between ISIN JS900123AL40 and JS900123BB58, which bond has the longer remaining term to maturity from the bulletin date?"},
    {"id": "C6", "name": "Comparison", "question": "Determine the difference in coupon rates between the Akbank 2026 (6.800%) and Vakifbank 2026 (6.500%) USD Eurobonds."},

    # --- SECTION 2: LOGIC & MATH ---
    {"id": "M1", "name": "Logic & Math", "question": "If a full taxpayer earns 450,000 TRY in Eurobond interest in 2025, by how much does this exceed the 330,000 TRY declaration threshold?"},
    {"id": "M2", "name": "Logic & Math", "question": "If the AKE fund total net value is 1.5 billion USD, what is the minimum dollar amount that must be invested in USD-denominated Turkish Eurobonds per the 80% rule?"},
    {"id": "M3", "name": "Logic & Math", "question": "Calculate the percentage difference between the Treasury Sukuk coupon (7.250%) and the Sisecam 2026 coupon (6.950%)."},
    {"id": "M4", "name": "Logic & Math", "question": "Calculate the total annual interest income in USD for a client holding 250,000 USD nominal of the Turkey 2047 bond (5.750% coupon)."},
    {"id": "M5", "name": "Logic & Math", "question": "If the maximum 1% commission is charged on a 15,000 USD nominal Eurobond transaction, what is the total commission fee?"},
    {"id": "M6", "name": "Logic & Math", "question": "Find the average 5-year CDS spread of Brazil and South Africa as of 24/12/2025 and determine how much lower it is than Turkey's CDS."},

    # --- SECTION 3: MULTI-HOP (Cross-Document) ---
    {"id": "H1", "name": "Multi-Hop", "question": "Based on the T+2 settlement rule in the HSBC document, if a trade is executed on the Gedik bulletin date (Dec 25, 2025), what is the specific settlement date?"},
    {"id": "H2", "name": "Multi-Hop", "question": "Find the 2025 tax declaration threshold in TRY and convert it to USD using an assumed exchange rate of 1 USD = 35 TRY."},
    {"id": "H3", "name": "Multi-Hop", "question": "Identify all USD government bonds in the 'Daily Transactions' table with a Moody's rating of 'Ba3' and a buy yield exceeding 6%."},
    {"id": "H4", "name": "Multi-Hop", "question": "Locate the coupon rate for Arcelik's EUR bond and determine its withholding tax rate based on the HSBC tax summary for individuals."},
    {"id": "H5", "name": "Multi-Hop", "question": "Compare the AKE fund's annual management fee (2.00%) with the maximum transaction commission rate (1.00%); which is higher?"},
    {"id": "H6", "name": "Multi-Hop", "question": "Find the bank bond with the highest coupon rate in the bulletin and identify which risk profile ('Low' to 'Very High') it belongs to according to the product guidelines."},

    # --- SECTION 4: FALLBACK & HALLUCINATION ---
    {"id": "F1", "name": "Fallback", "question": "What is the coupon rate of the Turkey 2062 Eurobond? If the data is missing, state it and provide the current stock price of Apple (AAPL)."},
    {"id": "F2", "name": "Fallback", "question": "Search the documents for 'Social Impact Bonds'. If not found, provide the definition of 'Country Risk' from the HSBC document instead."},
    {"id": "L1", "name": "Hallucination", "question": "Check the bulletin for the current trading price of 'Gedik Yatirim' stock. Do not provide a price if it is not explicitly in the text."},
    {"id": "L2", "name": "Hallucination", "question": "Confirm if the documents state that Turkish Treasury Eurobonds are covered by the 'Investor Compensation Center' up to 100,000 EUR."},
    {"id": "L3", "name": "Hallucination", "question": "Find the 'Loyalty Discount' for long-term holders in the HSBC fee section. If it does not exist, state the standard maximum commission."},

    # --- SECTION 5: TEMPORAL ---
    {"id": "T1", "name": "Temporal", "question": "Using the 'Active Turkey Bonds (USD)' table, describe the yield trend for the Turkey 2029 bond between 24/11/2025 and 24/12/2025."},
    {"id": "T2", "name": "Temporal", "question": "Calculate the number of years between the establishment of the Ak Portfoy AKE fund (Dec 2, 2010) and the bulletin date (Dec 25, 2025)."},
    {"id": "T3", "name": "Temporal", "question": "List all USD government bonds in the daily transactions table that have a maturity date in the year 2034 or later."},

    # --- SECTION 6: CATEGORICAL FILTERING ---
    {"id": "S1", "name": "Filtering", "question": "Extract all EUR-denominated bonds from the dataset and list their issuers and coupon rates."},
    {"id": "S2", "name": "Filtering", "question": "List all 'Subordinated' (Sermaye Benzeri) bonds found in the bulletin and include their respective bank names."},
    {"id": "S3", "name": "Filtering", "question": "Filter the list of government bonds to show only those with a minimum nominal transaction requirement of 200,000 USD."},
    {"id": "S4", "name": "Filtering", "question": "Identify which 'Non-Financial' private sector bonds have a maturity date specifically in the year 2026."},
    {"id": "S5", "name": "Filtering", "question": "List every bond in the 'Daily Transactions' tables that currently has an 'Ask Price' (Alis Fiyati) above 105.000."},

    # --- SECTION 7: POLICY & RISK ---
    {"id": "R1", "name": "Policy", "question": "Explain why the HSBC document advises against Eurobonds for investors who do not have 'emergency cash' reserves."},
    {"id": "R2", "name": "Policy", "question": "Detail the specific restrictions mentioned regarding the sale of these investment products to residents of the USA and Canada."},
    {"id": "R3", "name": "Policy", "question": "According to the 'Right of Withdrawal' section, can an investor cancel a Eurobond purchase after it has been executed?"},
    {"id": "R4", "name": "Policy", "question": "Define 'Liquidity Risk' as described in the fundamental risks section of the Eurobond documentation."},
    {"id": "R5", "name": "Policy", "question": "What is the tax status of Eurobond interest income for a 'limited taxpayer' individual (Dar mukellef)?"},

    # --- SECTION 8: COMPLEX SCENARIOS ---
    {"id": "X1", "name": "Scenario", "question": "An investor wants a bond with a coupon of at least 9.0%. List all government and private options that meet this criteria."},
    {"id": "X2", "name": "Scenario", "question": "Calculate the total TRY value of one semi-annual coupon payment for a 20,000 USD nominal holding of the Turkey 2030 bond (11.875% coupon), using 1 USD = 35 TRY."},
    {"id": "X3", "name": "Scenario", "question": "Compare the 'Average Dirty Price' (Ortalama Kirli Fiyat) of the Turkey 2030 bond with the Turkey 2047 bond."},
    {"id": "X4", "name": "Scenario", "question": "Is the 'Credit Risk' stated to be higher for corporate bonds or government bonds according to the HSBC Risk Profile?"},
    {"id": "X5", "name": "Scenario", "question": "Under what specific condition can the purchase price of a Eurobond be adjusted for inflation for capital gains tax purposes?"},
    {"id": "X6", "name": "Scenario", "question": "Who is the legal issuer responsible for the principal and interest payments of the Eurobonds listed in the HSBC document?"},
    {"id": "X7", "name": "Scenario", "question": "Describe the tax consequences of transferring Eurobonds from an individual account to a joint account according to the policy."},
    {"id": "X8", "name": "Scenario", "question": "Find the 'Days to Next Coupon' (Kupona Kalan Gun) for the Turkey 2034 bond (ISIN ending in AT75) as of the bulletin date."},
    {"id": "X9", "name": "Scenario", "question": "Which regulatory bodies are mentioned as the authorities supervising HSBC Bank A.S.'s banking and capital market activities?"}
]

# ==============================================================================
# GPT-4o MINI + PERSISTENT CHROMA RAG + BENCHMARK
# ==============================================================================

import time
import re
import json
import chromadb
import yfinance as yf
from openai import OpenAI
from google.colab import userdata

# =========================
# 1) GPT-4o MINI SETUP
# =========================
# API anahtarını Colab Secrets'tan "OPENAI_API_KEY" adıyla alır
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

def safe_gpt_generate(prompt: str, max_tokens: int = 500, retries: int = 5) -> str:
    last_err = None
    for attempt in range(retries):
        try:
            completion = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a financial analysis assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
                max_tokens=max_tokens,
            )
            text = completion.choices[0].message.content.strip()
            if text:
                return text

            time.sleep(1 + attempt)
            last_err = ValueError("Empty GPT response")
        except Exception as e:
            last_err = e
            time.sleep(2 + attempt)

    return f"ERROR: GPT-4o mini failed after retries. Last error: {last_err}"

# =========================
# 2) PERSISTENT CHROMA (Google Drive - agent folder)
# =========================
# Drive'ın /content/drive/My Drive/ adresine mount edildiğini varsayar
DB_PATH = "/content/drive/My Drive/agent_real/chroma_db_persistent"
COLLECTION_NAME = "eurobond_knowledge"

class ChromaRAG:
    def __init__(self, path: str, collection_name: str):
        try:
            self.client = chromadb.PersistentClient(path=path)
            self.collection = self.client.get_or_create_collection(name=collection_name)
            print(f"✅ Chroma connected | Location: {path} | Chunks: {self.collection.count()}")
        except Exception as e:
            print(f"❌ Chroma connection failed: {e}")

    def retrieve(self, query: str, k: int = 5) -> str:
        query = (query or "").strip()
        if not query:
            return "NO_INFORMATION_FOUND"
        res = self.collection.query(query_texts=[query], n_results=k)
        docs = res.get("documents", [[]])[0] or []
        if not docs:
            return "NO_INFORMATION_FOUND"
        return "\n".join(docs)

rag = ChromaRAG(DB_PATH, COLLECTION_NAME)

# =========================
# 3) TOOLS
# =========================
def search_knowledge_base(query: str) -> str:
    return rag.retrieve(query, k=5)

def calculator(expr: str) -> str:
    expr = (expr or "").replace(",", "").strip()
    if not re.fullmatch(r"[0-9+\-*/(). ]+", expr):
        return "ERROR"
    try:
        val = eval(expr)
        return str(val) if isinstance(val, (int, float)) else "ERROR"
    except:
        return "ERROR"

def last_close_price(ticker: str) -> str:
    t = (ticker or "").strip().upper()
    try:
        info = yf.Ticker(t).get_info()
        v = info.get("previousClose", None)
        return str(v) if v is not None else "NOT_FOUND"
    except:
        return "NOT_FOUND"

# =========================
# 4) RUN CASE
# =========================
def run_case(case):
    q = case["question"]

    # RAG ihtiyacı kontrolü
    need_rag = any(k in q.lower() for k in [
        "turkey", "eurobond", "tax", "threshold", "ake", "coupon", "yield", "ytm", "gedik"
    ])
    context = search_knowledge_base(q) if need_rag else "NO_INFORMATION_FOUND"

    # Dış fiyat kontrolü
    prices = {}
    for sym in ["AAPL", "TSLA"]:
        if sym in q:
            prices[sym] = last_close_price(sym)

    prices_block = "\n".join([f"{k}: {v}" for k, v in prices.items()]) if prices else "None"

    prompt = f"""
RULES:
- Use ONLY the provided context for internal data.
- If information is missing, explicitly state that it is missing from the context.
- For any math, show the step-by-step expression and the result.

Context:
{context}

External Prices:
{prices_block}

Question:
{q}

Final Answer:
""".strip()

    answer = safe_gpt_generate(prompt, max_tokens=450, retries=5)

    return {
        "id": case["id"],
        "name": case["name"],
        "question": q,
        "context_used": context,
        "external_prices": prices,
        "answer": answer
    }

# =========================
# 5) BENCHMARK EXECUTION
# =========================
def run_benchmark(cases):
    results = []
    print("\n" + "="*80 + "\nSTARTING GPT-4o MINI + CHROMA BENCHMARK\n" + "="*80)

    for case in cases:
        print(f"\nCASE {case['id']}: {case['question']}")
        result = run_case(case)
        results.append(result)
        print(f"ANSWER:\n{result['answer']}")

        # OpenAI Rate Limit koruması için kısa bekleme
        time.sleep(1)

    # Sonuçları Kaydet
    with open("/content/benchmark_results_gpt4o_mini.json", "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)

    print("\n✅ Results saved to /content/benchmark_results_gpt4o_mini.json")

# Benchmark'ı çalıştır
run_benchmark(benchmark_cases)



✅ Chroma connected | Location: /content/drive/My Drive/agent_real/chroma_db_persistent | Chunks: 300

STARTING GPT-4o MINI + CHROMA BENCHMARK

CASE C1: Calculate the spread in basis points between the Turkey 2036 buy yield (6.41%) and the US 10Y yield (4.13%) as of December 24, 2025.
ANSWER:
To calculate the spread in basis points between the Turkey 2036 buy yield and the US 10Y yield, we can use the following formula:

\[
\text{Spread} = \text{Turkey 2036 Yield} - \text{US 10Y Yield}
\]

Given:
- Turkey 2036 buy yield = 6.41%
- US 10Y yield = 4.13%

Now, substituting the values into the formula:

\[
\text{Spread} = 6.41\% - 4.13\%
\]

Calculating the spread:

\[
\text{Spread} = 6.41 - 4.13 = 2.28\%
\]

To convert the spread from percentage points to basis points, we multiply by 100:

\[
\text{Spread in basis points} = 2.28\% \times 100 = 228 \text{ basis points}
\]

Final Answer: 228 basis points

CASE C2: Compare the 5-year CDS spreads of Turkey, Brazil, and South Africa. Which count

# Drive Mount

In [5]:
import os
import shutil

# Try to unmount first (this often fails for colab drive mounts)
try:
    !fusermount -u /content/drive
except:
    pass

# Ensure the mount point is completely empty
if os.path.exists("/content/drive"):
    shutil.rmtree("/content/drive", ignore_errors=True)

# Recreate the empty directory
os.makedirs("/content/drive", exist_ok=True)

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

ValueError: Mountpoint must not already contain files

In [ ]:
import torch
torch.cuda.is_available()

False